In [ ]:
!pip cache purge
!pip install imapclient openai langchain
!pip install python-dotenv

!pip install -U langchain-community
!pip install langchain-openai
!pip install langchain-chroma

!pip install faiss-cpu
!pip install -U langchain-anthropic
!pip install langchain-huggingface
!pip install -q langgraph

Files removed: 389


## **Pipeline**:
get email  ->  summarize  ->  embeddings  ->  store FAISS  ->  query  ->  auto response mail  ->  auto post

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

load_dotenv("/content/API Key.env")
google_api_key = os.getenv('GEMINI_API_KEY')
hf_token = os.getenv("HF_TOKEN")

from huggingface_hub import login
login(token=hf_token)

import anthropic
claude_key = os.getenv("CLAUDE_API_KEY_TON")
claude_client = anthropic.Anthropic(api_key=claude_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Get emails "UNSEEN"**

In [ ]:
import imapclient
import email
from email.header import decode_header
import openai
import datetime
from bs4 import BeautifulSoup
import time

EMAIL = "nu330006@gmail.com"
PASSWORD = "kund pbor krgm rzhx"

mail = imapclient.IMAPClient("imap.gmail.com", ssl=True)
mail.login(EMAIL, PASSWORD)

mail.select_folder("INBOX")

days = 30
while True:
    days_ago = (datetime.datetime.now() - datetime.timedelta(days=days)).strftime("%d-%b-%Y")
    messages = mail.search(["SINCE", days_ago, "UNSEEN"])

    if len(messages) > 0 or days >= 600:
        break
    days += 10

print(f"🔍 Đang kiểm tra email chưa đọc trong {days} ngày qua...")


if mail.has_capability('SORT'):
    messages = mail.sort(["REVERSE DATE"], messages)
else:
    fetch_data = mail.fetch(messages, ['INTERNALDATE'])
    messages_with_dates = [(msg_id, fetch_data[msg_id][b'INTERNALDATE']) for msg_id in messages]
    messages_with_dates.sort(key=lambda item: item[1], reverse=True)
    messages = [item[0] for item in messages_with_dates]


🔍 Đang kiểm tra email chưa đọc trong 30 ngày qua...


In [ ]:
from typing_extensions  import TypedDict, Annotated, Optional, List, Any, Dict
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph, END
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace


class AgentState(TypedDict):
    emails: List[Dict[str, Any]]
    vector_db: Optional[any]
    query: Optional[str]
    relevant_emails: List[Dict[str, Any]]
    email_draft: List[str]
    history: List[str]

In [ ]:
from email.header import decode_header
import chardet
import re

def decode_email_content(content, encoding=None):
    if encoding:
        try:
            return content.decode(encoding)
        except UnicodeDecodeError:
            pass
    detected_encoding = chardet.detect(content)['encoding']
    if detected_encoding:
        return content.decode(detected_encoding)
    return content.decode('utf-8', errors='ignore')



def fetch_unread_mail(state: AgentState) -> AgentState:
    emails = []
    for idx, msg_id in list(enumerate(messages))[:50]:
        idx += 1
        raw_msg = mail.fetch(msg_id, ["RFC822"])[msg_id][b"RFC822"]
        msg = email.message_from_bytes(raw_msg)

        from_email = msg["From"]
        if isinstance(from_email, tuple):
            from_email = " ".join(str(part) for part in from_email)

        subject, encoding = decode_header(msg["Subject"])[0]
        subject = subject.decode(encoding) if encoding else subject

        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))

                if content_type == "text/plain" and "attachment" not in content_disposition:
                    body = part.get_payload(decode=True)
                    body = decode_email_content(body, part.get_content_charset())
                    break
                elif content_type == "text/html" and "attachment" not in content_disposition:
                    html_content = part.get_payload(decode=True)
                    html_content = decode_email_content(html_content, part.get_content_charset())

                    soup = BeautifulSoup(html_content, "lxml")

                    for tag in soup(['style', 'script', 'head', 'title', 'meta', 'link',
                                    'noscript', 'header', 'footer', 'table', 'tr', 'td',
                                    'th', 'tbody', 'thead', 'form', 'input']):
                        tag.decompose()

                    for element in soup.find_all(['p', 'div', 'br', 'li', 'h1', 'h2', 'h3']):
                        element.append('\n')

                    body = soup.get_text(separator='\n', strip=True)

                    body = re.sub(r'\n{3,}', '\n\n', body)
                    body = re.sub(r'[\xa0\u200b]+', ' ', body)
        else:
            body = msg.get_payload(decode=True)
            body = decode_email_content(body, msg.get_content_charset())

        emails.append({"id": idx, "from_email": from_email,
                        "subject": subject, "body": body, "status": "new"})

        return {"emails": emails}

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token=hf_token,
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
from google import genai
from langchain_core.tools import tool
from langchain.tools import Tool


def tool_summarize(email: str) -> str:
    """
    Summarize the email into its main sections: Purpose, Important Information, and Request
    """
    system_prompt = f"""You are a professional email summarization assistant. Summarize the email content into three sections::
    1. Main Purpose
    2. Key Information
    """
    response = claude_client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=600,
                system=system_prompt,
                messages=[{"role": "user", "content": email}]
                )
    return response.content[0].text


summarize_tool = Tool(
    name="summarize_email",
    description="Summarize the email into its main sections: Purpose, Important Information, and Request",
    func=tool_summarize
)

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token=hf_token,
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [summarize_tool]
chat_with_tools = chat.bind_tools(tools)

def summarize_emails(state: AgentState) -> AgentState:
    updated_emails = []
    for email in state["emails"]:
        message = HumanMessage(content=email["body"])
        summarized_body = chat_with_tools.invoke([message])

        updated_emails.append({**email, "body": summarized_body.content})

    return {"emails": updated_emails}

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


def store_data_to_vector_db(state: AgentState) -> AgentState:
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    emails = state["emails"]
    docs = [
        Document(
            page_content=email["body"],
            metadata={
                "id": email["id"],
                "from_email": email["from_email"],
                "subject": email["subject"],
                "original_body": email["body"]
            }
        ) for email in state["emails"]
    ]


    index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(docs)
    vector_store.save_local("/content/drive/MyDrive/faiss_index")

    return {
        "vector_db": vector_store
    }

In [ ]:
def query_emails(state: AgentState) -> AgentState:
    query = state.get("query")
    vector_db = state.get("vector_db")

    if not query or not vector_db:
        print("⚠️ Lỗi: Thiếu query hoặc vector_db")
        return {"relevant_emails": [], "query": query}

    try:
        docs_and_scores = vector_db.as_retriever(search_kwargs={"k": 10})
        docs = docs_and_scores.invoke(query)

        relevant_emails = []

        for doc in docs:
            relevant_emails.append({
                "id": doc.metadata["id"],
                "from_email": doc.metadata["from_email"],
                "subject": doc.metadata["subject"],
                "body": doc.metadata["original_body"],
            })

        return {
            "relevant_emails": relevant_emails,
            "query": query
        }

    except Exception as e:
        print(f"Error query: {str(e)}")
        return {"relevant_emails": [], "query": query}

In [ ]:
from langchain_anthropic import ChatAnthropic

def tool_generate_auto_reply(email_sender_name, email_body):
    """Create a brief response email with the given content"""
    llm = ChatAnthropic(model='claude-3-opus-20240229', anthropic_api_key=claude_key)
    prompt = f"""
    You are virtual assistant **Huynh Minh Tam**. Please write a reply to the following email:

    Email information:
    - Sender: {email_sender_name}
    - Email content: {email_body[:200]}

    Requirements:
    1. Start with "Dear [Sender's name]"
    2. Reply briefly and politely
    3. Mention the main topic of the email
    4. Sign: **Huynh Minh Tam**
    5. Sending email address: nu330006@gmail.com

    Reply:
    """
    response = llm.invoke(prompt)
    return response.content


auto_reply_tool = Tool(
    name="generate_auto_reply",
    description="Create a brief response email with the given content",
    func=tool_generate_auto_reply
)

tools.append(auto_reply_tool)

def generate_responses(state: AgentState) -> AgentState:
    responses = []
    for email in state["relevant_emails"]:
        # prompt = f"""Reply to this email from {email["from_email"]}:\n\n{email["body"]}"""
        # message = HumanMessage(content=prompt)
        # result = chat_with_tools.invoke([message])

        message = HumanMessage(content={
            "email_sender_name": email["from_email"],
            "email_body": email["body"]
        })
        result = chat_with_tools.invoke([message])
        responses.append(result.content)

    return {"email_draft": responses, "history": responses}

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_username = "nu330006@gmail.com"
smtp_password = "kund pbor krgm rzhx"

def send_email(sender_email: str, receiver_email: str, subject: str, body: str) -> bool:
    """
    Gửi email qua SMTP
    Trả về True nếu thành công, False nếu thất bại
    """
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject

    msg.attach(MIMEText(body, "plain", "utf-8"))
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(smtp_username, smtp_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        print(f"✅ Đã gửi email đến {receiver_email}")
        return True
    except Exception as e:
        print(f"❌ Lỗi khi gửi email đến {receiver_email}: {str(e)}")
        return False


def send_automation_emails(state: AgentState) -> AgentState:
    email_draft = state["email_draft"]
    for email, draft in zip(state["relevant_emails"], email_draft):
        received_email = email["from_email"]
        original_subject = email["subject"]

        send_email(
                sender_email=smtp_username,
                receiver_email=received_email,
                subject=original_subject,
                body=draft
            )
    return {**state, "email_draft": []}

In [ ]:
workflow = StateGraph(AgentState)


def check_emails(state: AgentState) -> str:
    if not state.get("emails"):
        return "no_emails"
    return "continue"

def check_emails_relevents(state: AgentState) -> str:
    if not state.get("relevant_emails"):
        return "no_emails"
    return "continue"


workflow.add_node("get_emails", fetch_unread_mail)
workflow.add_node("summarize_emails", summarize_emails)
workflow.add_node("store_data_to_vector_db", store_data_to_vector_db)
workflow.add_node("query_emails", query_emails)
workflow.add_node("generate_responses", generate_responses)
workflow.add_node("send_automation_emails", send_automation_emails)



workflow.add_edge(START, "get_emails")
workflow.add_conditional_edges(
    "get_emails",
    check_emails,
    {
        "continue": "summarize_emails",
        "no_emails": END
    }
)
workflow.add_edge("summarize_emails", "store_data_to_vector_db")
workflow.add_edge("store_data_to_vector_db", "query_emails")
workflow.add_conditional_edges(
    "query_emails",
    check_emails_relevents,
    {
        "continue": "generate_responses",
        "no_emails": END
    }
)
workflow.add_edge("query_emails", "generate_responses")
workflow.add_edge("generate_responses", "send_automation_emails")
workflow.add_edge("send_automation_emails", END)
error_edges = [
    "summarize_emails",
    "store_data_to_vector_db",
    "query_emails",
    "generate_responses",
    "send_automation_emails"
]

assistant = workflow.compile()

In [ ]:
initial_state = {
    "emails": [],
    "vector_db": None,
    "query": "show me email related to the topic about Education or Techonology",
    "relevant_emails": [],
    "email_draft": []
}

result = assistant.invoke(initial_state)
print(result)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-6808b9af-7a3828c0497315e12b308b54;f954de70-31a9-4d8a-ab75-08857a2dd2ee)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
from IPython.display import Image, display
display(Image(assistant.get_graph(xray=True).draw_mermaid_png()))